## Using Subqueries and APPLY

***

### Import Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random

import pyodbc

%matplotlib inline

pd.set_option('display.max_columns',None)
#pd.set_option('display.max_rows',None)
pd.set_option('display.width', 1000)
pd.option_context('float_format','{:.2f}'.format)

random.seed(0)
np.random.seed(0)
np.set_printoptions(suppress=True)

## Load Data from SQL database

### MSSQL

In [2]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DESKTOP-ARQLULI\SQLEXPRESS2019;'
                      'Database=AdventureWorksLT2012;'
                      'Trusted_Connection=yes;')

## Challenge 1: Retrieve Product Price Information

In [3]:
pd.read_sql_query("SELECT ProductID, Name, ListPrice \
                   FROM SalesLT.Product \
                   WHERE ListPrice > (SELECT AVG(UnitPrice) FROM SalesLT.SalesOrderDetail) \
                   ORDER BY ProductID;", conn)

,ProductID,Name,ListPrice
0,680,"HL Road Frame - Black, 58",1431.50
1,706,"HL Road Frame - Red, 58",1431.50
2,717,"HL Road Frame - Red, 62",1431.50
3,718,"HL Road Frame - Red, 44",1431.50
4,719,"HL Road Frame - Red, 48",1431.50
...,...,...,...
132,992,"Mountain-500 Black, 48",539.99
133,993,"Mountain-500 Black, 52",539.99
134,997,"Road-750 Black, 44",539.99
135,998,"Road-750 Black, 48",539.99


In [4]:
pd.read_sql_query("SELECT ProductID, Name, ListPrice \
                   FROM SalesLT.Product \
                   WHERE ProductID IN (SELECT ProductID from SalesLT.SalesOrderDetail WHERE UnitPrice < 100.00)\
                   AND ListPrice >= 100 \
                   ORDER BY ProductID;", conn)

,ProductID,Name,ListPrice
0,810,HL Mountain Handlebars,120.27
1,813,HL Road Handlebars,120.27
2,876,Hitch Rack - 4-Bike,120.00
3,894,Rear Derailleur,121.46
4,907,Rear Brakes,106.50
5,948,Front Brakes,106.50
6,996,HL Bottom Bracket,121.49


In [5]:
pd.read_sql_query("SELECT ProductID, Name, StandardCost, ListPrice, \
                    (SELECT AVG(UnitPrice) \
                    FROM SalesLT.SalesOrderDetail AS SOD \
                    WHERE P.ProductID = SOD.ProductID) AS AvgSellingPrice \
                    FROM SalesLT.Product AS P\
                    ORDER BY P.ProductID;", conn)

,ProductID,Name,StandardCost,ListPrice,AvgSellingPrice
0,680,"HL Road Frame - Black, 58",1059.3100,1431.50,NaN
1,706,"HL Road Frame - Red, 58",1059.3100,1431.50,NaN
2,707,"Sport-100 Helmet, Red",13.0863,34.99,20.9940
3,708,"Sport-100 Helmet, Black",13.0863,34.99,20.6441
4,709,"Mountain Bike Socks, M",3.3963,9.50,NaN
...,...,...,...,...,...
290,995,ML Bottom Bracket,44.9506,101.24,NaN
291,996,HL Bottom Bracket,53.9416,121.49,72.8940
292,997,"Road-750 Black, 44",343.6496,539.99,323.9940
293,998,"Road-750 Black, 48",343.6496,539.99,323.9940


In [6]:
pd.read_sql_query("SELECT ProductID, Name, StandardCost, ListPrice, \
                 (SELECT AVG(UnitPrice)\
                  FROM SalesLT.SalesOrderDetail AS SOD \
                  WHERE P.ProductID = SOD.ProductID) AS AvgSellingPrice \
                  FROM SalesLT.Product AS P \
                  WHERE StandardCost > \
                  (SELECT AVG(UnitPrice) \
                  FROM SalesLT.SalesOrderDetail AS SOD \
                  WHERE P.ProductID = SOD.ProductID)\
                  ORDER BY P.ProductID;",conn)

,ProductID,Name,StandardCost,ListPrice,AvgSellingPrice
0,712,AWC Logo Cap,6.9223,8.99,5.3740
1,714,"Long-Sleeve Logo Jersey, M",38.4923,49.99,29.9940
2,715,"Long-Sleeve Logo Jersey, L",38.4923,49.99,29.7440
3,716,"Long-Sleeve Logo Jersey, XL",38.4923,49.99,29.9940
4,717,"HL Road Frame - Red, 62",868.6342,1431.50,858.9000
5,718,"HL Road Frame - Red, 44",868.6342,1431.50,858.9000
6,722,"LL Road Frame - Black, 58",204.6251,337.22,202.3320
7,738,"LL Road Frame - Black, 52",204.6251,337.22,202.3320
8,792,"Road-250 Red, 58",1554.9479,2443.35,1466.0100
9,793,"Road-250 Black, 44",1554.9479,2443.35,1466.0100


## Challenge 2: Retrieve Customer Information

In [7]:
pd.read_sql_query("SELECT SOH.SalesOrderID, SOH.CustomerID, CI.FirstName, CI.LastName, SOH.TotalDue \
                   FROM SalesLT.SalesOrderHeader AS SOH \
                   CROSS APPLY dbo.ufnGetCustomerInformation(SOH.CustomerID) AS CI \
                   ORDER BY SOH.SalesOrderID;",conn)

,SalesOrderID,CustomerID,FirstName,LastName,TotalDue
0,71774,29847,David,Hodgson,972.7850
1,71776,30072,Andrea,Thomsen,87.0851
2,71780,30113,Raja,Venugopal,42452.6519
3,71782,29485,Catherine,Abel,43962.7901
4,71783,29957,Kevin,Liu,92663.5609
5,71784,29736,Terry,Eminhizer,119960.8240
6,71796,29660,Anthony,Chor,63686.2708
7,71797,29796,Jon,Grande,86222.8072
8,71815,30089,Michael John,Troyer,1261.4440
9,71816,30027,Joseph,Mitzner,3754.9733


In [8]:
pd.read_sql_query("SELECT CA.CustomerID, CI.FirstName, CI.LastName, A.AddressLine1, A.City \
                   FROM SalesLT.Address AS A \
                   JOIN SalesLT.CustomerAddress AS CA \
                   ON A.AddressID = CA.AddressID \
                   CROSS APPLY dbo.ufnGetCustomerInformation(CA.CustomerID) AS CI \
                   ORDER BY CA.CustomerID;", conn)

,CustomerID,FirstName,LastName,AddressLine1,City
0,29485,Catherine,Abel,57251 Serene Blvd,Van Nuys
1,29486,Kim,Abercrombie,Tanger Factory,Branch
2,29489,Frances,Adams,6900 Sisk Road,Modesto
3,29490,Margaret,Smith,Lewiston Mall,Lewiston
4,29492,Jay,Adams,Blue Ridge Mall,Kansas City
...,...,...,...,...,...
412,30113,Raja,Venugopal,Burgess Hill,West Sussex
413,30115,Dora,Verdad,Suite 2502 410 Albert Street,Waterloo
414,30116,Wanda,Vernon,Ontario Mills,Ontario
415,30117,Robert,Vessa,72540 Blanco Rd.,San Antonio


In [9]:
conn.close()

#### Python code done by Dennis Lam